In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# tool create
@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a * b

In [4]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [5]:
multiply.name, multiply.description, multiply.args

('multiply',
 'Given 2 numbers a and b this tool returns their product',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}})

In [6]:
# tool binding
llm = ChatOpenAI()

In [7]:
llm_with_tools = llm.bind_tools([multiply])

In [8]:
# Tool Calling
llm_with_tools.invoke('Hi, how are you?')

AIMessage(content="Hello! I'm here and ready to help. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 60, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C92BrvIxYkD5In2FPkIpO5KflZyW2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b15e08a5-1e30-402d-9feb-532574681653-0', usage_metadata={'input_tokens': 60, 'output_tokens': 18, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
llm_with_tools.invoke('can you multiply 3 with 10')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PvBF5CZstRxxFoNef3DTLesC', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 62, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C92BsvayuT4bH2zoORTGFgqFtLzdy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--12a21af1-051f-4bf1-b638-4149f0560707-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_PvBF5CZstRxxFoNef3DTLesC', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_tokens': 17, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [10]:
llm_with_tools.invoke('can you multiply 3 with 10').tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'call_SCNruikO1jaMKlI2vPOuFFGN',
  'type': 'tool_call'}]

In [11]:
query = HumanMessage('can you multiply 3 with 1000?')

In [12]:
message = [query]

In [13]:
message

[HumanMessage(content='can you multiply 3 with 1000?', additional_kwargs={}, response_metadata={})]

In [14]:
result = llm_with_tools.invoke(message)

In [15]:
# llm_with_tools.invoke('can you multiply 3 with 10?').tool_calls

In [16]:
# result = llm_with_tools.invoke('can you multiply 3 with 10?')

In [17]:
message.append(result)

In [18]:
message

[HumanMessage(content='can you multiply 3 with 1000?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eAvUGNInYFtKFBku3blRj7DR', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 64, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C92Bt3o8r1kB7f6c46BUJmohmIIFS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--60b9906b-afbb-46c2-ba88-6cd01425ac2e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_eAvUGNInYFtKFBku3blRj7DR', 'type': 'tool_call'}], usage_metadata={'input_tok

In [28]:
print(result.tool_calls[0])

{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_eAvUGNInYFtKFBku3blRj7DR', 'type': 'tool_call'}


In [20]:
# tool execution
tool_result = multiply.invoke(result.tool_calls[0])

In [21]:
message.append(tool_result)

In [29]:
message

[HumanMessage(content='can you multiply 3 with 1000?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eAvUGNInYFtKFBku3blRj7DR', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 64, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C92Bt3o8r1kB7f6c46BUJmohmIIFS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--60b9906b-afbb-46c2-ba88-6cd01425ac2e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_eAvUGNInYFtKFBku3blRj7DR', 'type': 'tool_call'}], usage_metadata={'input_tok

In [23]:
llm_with_tools.invoke(message).content

'The product of 3 and 1000 is 3000.'